# Purpose
## Background
Have you ever tried to use RPG Maker autotiles/terrain tiles in TileD or another mapping tool?

It doesn't work great.

The reason is because RPG Maker's autotiles are actually subdivided into 4 'mini-tiles' which are composited back together into full-sized tiles as you paint.

TileD can *approximate* this, but doesn't fully understand where each part of each tile goes. As such, it picks randomly for each of the edge and center peices, and doesn't create perfect patterns. If you already noticed this, well, that's probably why you're here.

## This Program
That's where this program comes in.

It's purpose is to take RPG Maker tilesets (which are 9 tiles), take them apart, composite them, and output the 47-tile tilests that work better in basically every other program.

You don't have to *write* any code to use it, but you do have to upload files, write in the names of those files, and change some other parameters. Then you have to go down a long list of 'code blocks' and run them with the little play buttons on the left of each block, one after the other.

If it all goes well, it'll output your finished 47-tile tilesets at the end.

# Set your Parameters
If all goes well, this will be the only code you actually edit. Once you've put in all your parameters and uploaded your file, click the little play button on the code.

**filename** should be the name of the file you are reading in, minus the file extension. Make sure to actually add the file to the collab! You can do it by clicking on the left to see 'Files' and then clicking to upload to the currently displayed directory. You may need to refresh to see your files.

**tileSize** should be set to 48, 32, 16, etc based on what the size of the tile is.

**startingPixel** is almost always going to be (0,0), because most of the time your RPGMaker autotiles start right up against the upper left side of the tilesheet. It's (width x height)

**terrainTypes** tells me how many groups of autotiles exist in each direction on this tileset. If there's just one autotile, type (1,1). If there's 8 autotiles across and 4 autotiles down, type (8,4).

**Note: ** Your autotiles have to all be all bunched together in a continous group, a rectangle. If they're scattered all over the place, well, you can either run this utility on a bunch of things that *aren't* autotiles in the middle, or you can fool around with startingPixel to pick out the autotiles in the middle that aren't conforming.


In [6]:
#Enter Primary Properties
fileName = "Sea"
tileSize = 32

startingPixel = (0,0)
terrainTypes = (8,1)

### Calculate Derivative Properties
At this point, assuming you put in the parameters properly, and clicked the play button to run the code, and got no errors...

...you should just be clicking on all the play buttons to come, cross your fingers, and pray nothing goes wrong.

In [7]:
#Computing Derivative Properties Based on What You Entered
totalTerrainTypes = terrainTypes[0] * terrainTypes[1]

terrainPixelSize = (2*tileSize, 3*tileSize)
totalPixelSize = (terrainTypes[0] * terrainPixelSize[0], terrainTypes[1] * terrainPixelSize[1])

saveWidth = tileSize*8
saveHeight = tileSize*6
halfTile = int(tileSize/2)

#File path names
loadPath = fileName + ".png"
savePaths = [None] * totalTerrainTypes

for i in range(0, totalTerrainTypes):
    savePaths[i] = fileName + "_output" + str(i) + ".png"


# Define Tile Locations

## Define Source Tile Locations

In [8]:
sourceTiles = [None]*(4*5)

m = 0

#Explore the vertical dimension second
for k in range(0,3):

    #Explore the horizontal dimension first
    for h in range(0,2):

        #Just ignore that upper left square
        if (k==0) and (h == 0):
            continue

        #explore the vertical dimension second
        for j in range(0,2):

            #explore the horizontal dimension first
            for i in range(0,2):
                startX = i*halfTile + h*tileSize
                endX = startX + halfTile
                startY = j*halfTile + k*tileSize
                endY = startY + halfTile

                sourceTiles[m] = (startX, startY, endX, endY)
                m +=1


## Define Output Locations

In [9]:
outputTiles = [None]*12
for j in range(0,12):
    outputTiles[j] = [None]*16
    for i in range(0,16):
        outputTiles[j][i] = 0

outputTiles[0] = [4,5,8,9,  16,13,16,13,    4,5,8,9,    16,1,0,13]
outputTiles[1] = [6,7,10,11,    10,3,2,7,   6,3,2,11,   2,3,2,3]
outputTiles[2] = [12,13,16,17,  16,1,0,13,  12,1,0,17,  0,1,0,1]
outputTiles[3] = [14,15,18,19,  10,7,10,7,  14,15,18,19,    10,3,2,7]

outputTiles[4] = [12,13,8,5,    12,13,8,5,  12,1,8,5,   12,1,8,5]
outputTiles[5] = [6,7,10,7,    6,3,2,13,   6,7,10,3,   6,3,2,3]
outputTiles[6] = [16,13,16,17,  16,1,0,17,  0,13,16,17, 0,1,0,17]
outputTiles[7] = [18,15,10,11,   18,15,10,11, 18,15,2,11, 18,15,2,11]

outputTiles[8] = [16,13,0,13,   4,9,4,5,    8,5,8,9,    12,17,0,13]
outputTiles[9] = [2,3,2,7,  6,11,14,15, 18,15,18,19,    6,11,10,3]
outputTiles[10] = [16,1,0,1,    12,17,16,13,    4,9,0,1,    None, None, 16, 1]
outputTiles[11] = [10,3,10,7,   14,19,10,7, 14,19,2,3,      None, None, 2,7]


# Let's Begin Converting


## Open Images

In [10]:
from PIL import Image

# Open File
toEdit = Image.open(loadPath).convert("RGBA")

# Create New File
allSaves = [None] * totalTerrainTypes

## Convert Images

In [11]:
m = 0
for k in range(0, terrainTypes[0]):
    for h in range(0, terrainTypes[1]):

        #Create an image to save to
        toSave = Image.new(mode="RGBA", size=(saveWidth, saveHeight))

        for j in range(0,12):
            for i in range(0,16):

                #Look up which tile is meant to be in this slot
                which = outputTiles[j][i]

                #Don't print any tile if the key is 'None'
                if which == None:
                    continue

                #Get the rectangle bounding the tile that's meant to be in this slot. It's [inclusive,exclusive)
                copyBox = sourceTiles[which]

                #Adjusting that rectangle by starting pixel size and based on which terrain we're converting
                source = (  copyBox[0] + startingPixel[0] + k*terrainPixelSize[0],
                            copyBox[1] + startingPixel[1] + h*terrainPixelSize[1],
                            copyBox[2] + startingPixel[0] + k*terrainPixelSize[0],
                            copyBox[3] + startingPixel[1] + h*terrainPixelSize[1])

                #Get the location we'll be pasting the tile to.
                pastePoint = (i*halfTile, j*halfTile)

                #Debugging
                #print(str(pastePoint) + ", tile: " + str(which) + ", i: " + str(i) + ", j:" + str(j))

                #Paste the tile (no, you don't just use 'paste' or 'copy', you have to use this unweildy monster)
                toSave.alpha_composite(im=toEdit,dest=pastePoint, source=source)

        # Show off the result so we can be sure we did it right.
        allSaves[m] = toSave
        m+=1

## Save to File

In [12]:
# Show the File (this process is actually extremely slow, so I discincluded it)
#for i in range(0, totalTerrainTypes):
    #allSaves[i].show()

for i in range(0, totalTerrainTypes):
    allSaves[i].save(fp=savePaths[i])

# End
Hit 'REFRESH' on the File Explorer on the left to see your outputted files...

## Troubleshooting
...How did it go? Did the converter work for you?

Sadly, if it malfunctioned, you're going to have to save a version and try to fix it yourself!

I cannot teach you Python, Pillow Collab, or Jupyter notebooks--I can't offer support for this tool.

All I can do is put this tool online and make it available so that hopefully some tech-artists find it, and then don't have to do a bunch of manual work copy+pasting things in Aesprite, Photoshop, or Gimp.

### Running Locally on your Own Machine
Python is one of those languages where you write very little code to leverage powerful libraries written by someone else in order to automate boring tasks like copying and pasting graphics in a programmatic pattern.

So anyone who wants to work in Python has to get used to the idea that, for every problem they have, there's a library for that. I mean, there's more than one, but there'd definitely *one*, and usually one everyone knows and has made a million tutorials for.

When it comes to quickly editing images, that library is [Pillow](https://pypi.org/project/pillow/). That's what that 'import Image from PIL' line was all about.

If you want to run a verison of this collab on your own computer, you'll need to have a development environment-- I use [Visual Studio Code](https://www.google.com/search?q=visual+studio+code&sca_esv=598988451&rlz=1C1CHZL_enHK710HK710&ei=lCqnZbmZILOt5NoPucKp8AQ&ved=0ahUKEwi5s-OLoOODAxWzFlkFHTlhCk4Q4dUDCBA&uact=5&oq=visual+studio+code&gs_lp=Egxnd3Mtd2l6LXNlcnAiEnZpc3VhbCBzdHVkaW8gY29kZTIQEAAYgAQYigUYQxixAxiDATIKEAAYgAQYigUYQzILEAAYgAQYigUYkQIyCxAAGIAEGIoFGJECMgsQABiABBiKBRiRAjIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAESOQBUABYAHAAeAGQAQCYAWagAWaqAQMwLjG4AQPIAQD4AQHiAwQYACBB&sclient=gws-wiz-serp) --and to have installed [Python](https://www.python.org/) itself...

You will either need to copy+paste each of the code blocks one by one into your python file or you can run the whole collab locally, like me, by working with [Jupyter Notebooks](https://code.visualstudio.com/docs/datascience/jupyter-notebooks).

You'll need to install something to manage python packages/libraries, if you don't already have one. Options include '[pip](https://colab.research.google.com/drive/1pK-jr_15GjqxKqSUtrmQuLVsSE2_IQq_#scrollTo=D0ukuux044Re&line=14&uniqifier=1)' which I'm pretty sure is straight up bundled with python these days, or a version of '[conda](https://conda.io/projects/conda/en/latest/user-guide/install/index.html).'

The process of setting up python can be briefly overwhelming because it feels like there's a lot of peices just to get started. This can be disheartening if things go wrong and, for example, VSCode can't find your damn Python path variable. Grr! Then you have to set it manually! But if things go wrong on your first attempt, don't give up, you'll get it!
